In [ ]:
# Copyright (c) Meta Platforms, Inc. and affiliates.

# SA-CO/Gold データの視覚化 (Visualization)

このノートブックでは、SA-CO/Goldデータセットの構造を理解し、アノテーションデータを視覚化する方法を示します。
SA-COデータセットは、セグメンテーションタスクのための大規模なデータセットです。

## Google Colab セットアップ

Google Colabで実行する場合は、以下のセルを実行してください。
これにより、必要なライブラリのインストールと、データの読み込み準備が行われます。

In [ ]:
# Google Colab環境のセットアップ
try:
    import google.colab
    IN_COLAB = True
    print("Google Colab環境で実行中")
except ImportError:
    IN_COLAB = False
    print("ローカル環境で実行中")

if IN_COLAB:
    # 1. Google Driveをマウント (推奨)
    # 自分のデータやコードを使用する場合は、Google Driveにアップロードしてマウントします
    from google.colab import drive
    drive.mount('/content/drive')
    
    # 2. SAM3のインストールとパス設定
    import os
    import sys
    
    # Google Drive内のSAM3ディレクトリのパス
    # ※ご自身の環境に合わせてパスを変更してください
    DRIVE_SAM3_PATH = "/content/drive/MyDrive/sam3"
    
    if os.path.exists(DRIVE_SAM3_PATH):
        print(f"Google Drive内のSAM3が見つかりました: {DRIVE_SAM3_PATH}")
        os.chdir(DRIVE_SAM3_PATH)
        print("カレントディレクトリを変更しました。")
        
        # 依存関係のインストール
        print("依存関係をインストールしています...")
        # Numpy 2.0との互換性問題を回避するためにバージョンを固定
        !pip install -q "numpy<2.0"
        !pip install -q -e .
        !pip install -q pycocotools
        
    else:
        print(f"Google Drive内に {DRIVE_SAM3_PATH} が見つかりませんでした。")
        print("GitHubからSAM3をクローンしてインストールします...")
        
        # GitHubからクローン
        if not os.path.exists("/content/sam3"):
            !git clone https://github.com/facebookresearch/sam3.git /content/sam3
            
        os.chdir("/content/sam3")
        print("カレントディレクトリを /content/sam3 に変更しました。")
        
        # 依存関係のインストール
        # Numpy 2.0との互換性問題を回避するためにバージョンを固定
        !pip install -q "numpy<2.0"
        !pip install -q -e .
        !pip install -q pycocotools
    
    print("セットアップが完了しました。")
    print(f"現在の作業ディレクトリ: {os.getcwd()}")

### データの使用について

- **Google Driveを使用する場合**: `GT_DIR` や `PRED_DIR` には `/content/drive/MyDrive/...` から始まるパスを指定してください。
- **GitHubからクローンした場合**: 左側のファイルブラウザから `/content/sam3` 内を確認できます。データは別途アップロードが必要です。

In [ ]:
# Google Colab環境のセットアップ
try:
    import google.colab
    IN_COLAB = True
    print("Google Colab環境で実行中")
except ImportError:
    IN_COLAB = False
    print("ローカル環境で実行中")

if IN_COLAB:
    # 1. Google Driveをマウント (推奨)
    # 自分のデータやコードを使用する場合は、Google Driveにアップロードしてマウントします
    from google.colab import drive
    drive.mount('/content/drive')
    
    # 2. SAM3のインストールとパス設定
    import os
    import sys
    
    # Google Drive内のSAM3ディレクトリのパス
    # ※ご自身の環境に合わせてパスを変更してください
    DRIVE_SAM3_PATH = "/content/drive/MyDrive/sam3"
    
    if os.path.exists(DRIVE_SAM3_PATH):
        print(f"Google Drive内のSAM3が見つかりました: {DRIVE_SAM3_PATH}")
        os.chdir(DRIVE_SAM3_PATH)
        print("カレントディレクトリを変更しました。")
        
        # 依存関係のインストール
        print("依存関係をインストールしています...")
        !pip install -q -e .
        !pip install -q pycocotools
        
    else:
        print(f"Google Drive内に {DRIVE_SAM3_PATH} が見つかりませんでした。")
        print("GitHubからSAM3をクローンしてインストールします...")
        
        # GitHubからクローン
        if not os.path.exists("/content/sam3"):
            !git clone https://github.com/facebookresearch/sam3.git /content/sam3
            
        os.chdir("/content/sam3")
        print("カレントディレクトリを /content/sam3 に変更しました。")
        
        # 依存関係のインストール
        !pip install -q -e .
        !pip install -q pycocotools
    
    print("セットアップが完了しました。")
    print(f"現在の作業ディレクトリ: {os.getcwd()}")

### データの使用について

- **Google Driveを使用する場合**: `GT_DIR` や `PRED_DIR` には `/content/drive/MyDrive/...` から始まるパスを指定してください。
- **GitHubからクローンした場合**: 左側のファイルブラウザから `/content/sam3` 内を確認できます。データは別途アップロードが必要です。

## 環境設定

Google Colabを使用しているかどうかを設定し、必要なライブラリをインストールします。

In [ ]:
using_colab = False

In [ ]:
if using_colab:
    import torch
    import torchvision
    print("PyTorch version:", torch.__version__)
    print("Torchvision version:", torchvision.__version__)
    print("CUDA is available:", torch.cuda.is_available())
    import sys
    !{sys.executable} -m pip install opencv-python matplotlib scikit-learn
    !{sys.executable} -m pip install 'git+https://github.com/facebookresearch/sam3.git'

## ライブラリのインポート

データ処理と視覚化に必要なライブラリをインポートします。

In [ ]:
import os
from glob import glob

import numpy as np
import sam3.visualization_utils as utils

from matplotlib import pyplot as plt

COLORS = utils.pascal_color_map()[1:]

## 1. データの読み込み (Load the data)

データセットのパスを指定し、アノテーションファイルを読み込みます。
ユーザーは `ANNOT_DIR` と `IMG_DIR` を自身の環境に合わせて変更する必要があります。

In [ ]:
# データパスの準備
ANNOT_DIR = None # ここにアノテーションのパスを入力
IMG_DIR = None # ここに画像のパスを入力

# SA-CO/Goldアノテーションファイルの読み込み
annot_file_list = glob(os.path.join(ANNOT_DIR, "*gold*.json"))
annot_dfs = utils.get_annot_dfs(file_list=annot_file_list)

読み込まれたアノテーションファイルを確認します。

In [ ]:
annot_dfs.keys()

## 2. データ形式の例 (Examples of the data format)

読み込んだデータの構造を確認します。
`gold_fg_sports_equipment_merged_a_release_test` データセットを例として使用します。

In [ ]:
annot_dfs["gold_fg_sports_equipment_merged_a_release_test"].keys()

### データセット情報 (Info)

データセットのメタデータを確認します。

In [ ]:
annot_dfs["gold_fg_sports_equipment_merged_a_release_test"]["info"]

### 画像情報 (Images)

データセットに含まれる画像の情報（ファイル名、IDなど）を確認します。

In [ ]:
annot_dfs["gold_fg_sports_equipment_merged_a_release_test"]["images"].head(3)

### アノテーション情報 (Annotations)

個々のアノテーションデータ（セグメンテーションマスク、バウンディングボックスなど）を確認します。

In [ ]:
annot_dfs["gold_fg_sports_equipment_merged_a_release_test"]["annotations"].head(3)

## 3. データの視覚化 (Visualize the data)

実際の画像とアノテーションマスクを視覚化して確認します。
ランダムに画像と名詞句のペアを選択し、対応する画像とマスクを表示します。

In [ ]:
# ターゲットデータセットの選択
target_dataset_name = "gold_fg_food_merged_a_release_test"

import cv2
from pycocotools import mask as mask_util
from collections import defaultdict

# GTアノテーションをimage_idでグループ化
gt_image_np_pairs = annot_dfs[target_dataset_name]["images"]
gt_annotations = annot_dfs[target_dataset_name]["annotations"]

gt_image_np_map = {img["id"]: img for _, img in gt_image_np_pairs.iterrows()}
gt_image_np_ann_map = defaultdict(list)
for _, ann in gt_annotations.iterrows():
    image_id = ann["image_id"]
    if image_id not in gt_image_np_ann_map:
        gt_image_np_ann_map[image_id] = []
    gt_image_np_ann_map[image_id].append(ann)

positiveNPs = common_image_ids = [img_id for img_id in gt_image_np_map.keys() if img_id in gt_image_np_ann_map and gt_image_np_ann_map[img_id]]
negativeNPs = [img_id for img_id in gt_image_np_map.keys() if img_id not in gt_image_np_ann_map or not gt_image_np_ann_map[img_id]]

num_image_nps_to_show = 10
fig, axes = plt.subplots(num_image_nps_to_show, 3, figsize=(15, 5 * num_image_nps_to_show))
for idx in range(num_image_nps_to_show):
    rand_idx = np.random.randint(len(positiveNPs))
    image_id = positiveNPs[rand_idx]
    noun_phrase = gt_image_np_map[image_id]["text_input"]
    img_rel_path = gt_image_np_map[image_id]["file_name"]
    full_path = os.path.join(IMG_DIR, f"{img_rel_path}")
    img = cv2.imread(full_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    gt_annotation = gt_image_np_ann_map[image_id]

    def display_image_in_subplot(img, axes, row, col, title=""):
        axes[row, col].imshow(img)
        axes[row, col].set_title(title)
        axes[row, col].axis('off')


    noun_phrases = [noun_phrase]
    annot_masks = [mask_util.decode(ann["segmentation"]) for ann in gt_annotation]

    # 画像を表示
    display_image_in_subplot(img, axes, idx, 0, f"{noun_phrase}")

    # 白背景上に全てのマスクを表示
    all_masks = utils.draw_masks_to_frame(
        frame=np.ones_like(img)*255, masks=annot_masks, colors=COLORS[: len(annot_masks)]
    )
    display_image_in_subplot(all_masks, axes, idx, 1, f"{noun_phrase} - Masks only")

    # 画像上にマスクを重ねて表示
    masked_frame = utils.draw_masks_to_frame(
        frame=img, masks=annot_masks, colors=COLORS[: len(annot_masks)]
    )
    display_image_in_subplot(masked_frame, axes, idx, 2, f"{noun_phrase} - Masks overlaid")
